In [1]:
import openslide
from openslide import deepzoom

In [2]:
def extract_patches(osr_obj, tile_size):
    """Description: Uses openslide to extract patches at closest zoom size
    INPUT: an openslide object (the .svs image)
    OUTPUT: a list of tiles as image objects
    """
    list_of_tiles = []
    
    t = openslide.deepzoom.DeepZoomGenerator(osr_obj, tile_size=tile_size, overlap=0, limit_bounds=False)
    max_level = t.level_count
    
    addresses = t.level_tiles[max_level-1] # zoomed in to furthest amount
    
    for x_coord in range(addresses[0]):
        for y_coord in range(addresses[1]):
            list_of_tiles.append(t.get_tile(max_level - 1, (x_coord, y_coord)))
    
    return list_of_tiles

In [2]:
def check_if_white(pixel):
    """DESCRIPTION: Checks to see if a given pixel is an off-white or white pixel
    INPUT: 3x1 tuple, where each element is in range [0,255]
    OUTPUT: 1 : is a whitespace
            0 : is not a whitespace
    """
    for x in pixel:
        if x < 230:
            return 0
    return 1

In [2]:
def determine_quality(img, WHITESPACE_CUTOFF):
    """Description: Determines quality of a histology tile image
    INPUT: Tile (Image object)
    OUTPUT: 0 : too much whitespace
            1 : acceptable image
    
    """
    p = img.load()
    img_size = img.size
    
    total = 0
    
    for y in range(img_size[1]):
        for x in range(0,img_size[0],3):
            if check_if_white(p[x,y]):
                total += 1

    if total / (img_size[0] * img_size[1] / 3) > WHITESPACE_CUTOFF:
        return 0 # too much whitespace
    else:
        return 1 # acceptable

In [4]:
def pad(i, n):
    """ PADS AN INTEGER
    INPUT: i : [int] number to be padded
           n : [int] total length of padded number
    OUTPUT: str
    """
    i = str(i)
    l = len(i)
    
    if l > n:
        raise ValueError('Integer is bigger than padding specifications')
    else:
        return ('0' * (n-l)) + i